**Задание:** Предсказание просрочки / просрочки 30 дней / просрочки 60 дней на основании данных финансовой отчетности компаний-контрагентов и других показателей

**Целевая переменная:** pdz_fact, pdz_30, pdz_60
В 2021 году показателя максимальное пдз нет, но есть разбивка по категориям была просрочка до 30, до 90, до 365 дней
Мы предполагаем использовать данные 2021 как валидационный дататест и целевая переменная там будет аналогичной, но построена специфически.

**Метрика качества:** 

**Описание датасета:**

Представлены 3 файла 
 - agents2019, agents2020, agents2021

В них представлены фин показатели за последние 3 года, а также:

Сред. ПДЗ, Кол-во просрочек свыше 5-ти дней, Общая сумма ПДЗ свыше 5-ти дней, Кол-во раз ПДЗ за 2019 год, шт.
В 2020 и 2021 годах - некие фактические показатели (1-60 штук), которые сводятся к итоговой величине
 

In [2]:
import numpy as np
import pandas as pd
import pickle   # сохранение модели

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree, DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import r2_score as r2, mean_squared_error as mse, roc_auc_score, recall_score

from sklearn import svm
from sklearn.svm import SVR

import statsmodels.api as sm
import statsmodels.formula.api as smf

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from scipy.stats import norm
from scipy import stats

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import warnings
warnings.filterwarnings('ignore')

import matplotlib.image as img
from scipy import stats
from IPython.display import Image

In [3]:
#track to files
data_2019 = pd.read_csv('agents2019.csv', sep=',')
data_2020 = pd.read_csv('agents2020.csv', sep=',')
data_2021 = pd.read_csv('agents2021.csv', sep=',')

In [4]:
all_columns = ['Наименование ДП', 'Макс. ПДЗ за 2019 год, дней', 'Сред. ПДЗ за 2019 год, дней', 
                       'Кол-во просрочек свыше 5-ти дней за 2019 год, шт.', 'Общая сумма ПДЗ свыше 5-ти дней за 2019 год, руб.', 
                      'Кол-во раз ПДЗ за 2019 год, шт.',
               'Макс. ПДЗ за 2020 год, дней', 'Сред. ПДЗ за 2020 год, дней', 'Кол-во просрочек свыше 5-ти дней за 2020 год, шт.',
               'Общая сумма ПДЗ свыше 5-ти дней за 2020 год, руб.', 'Кол-во раз ПДЗ за 2020 год, шт.',
               '2016, Основные средства , RUB', '2017, Основные средства , RUB', '2018, Основные средства , RUB', 
               '2019, Основные средства , RUB', '2020, Основные средства , RUB',
               '2016, Оборотные активы, RUB', '2017, Оборотные активы, RUB', '2018, Оборотные активы, RUB', 
               '2019, Оборотные активы, RUB', '2020, Оборотные активы, RUB', 
               '2016, Капитал и резервы, RUB', '2017, Капитал и резервы, RUB', '2018, Капитал и резервы, RUB', 
               '2019, Капитал и резервы, RUB', '2020, Капитал и резервы, RUB', 
               '2016, Заёмные средства (долгосрочные), RUB', '2017, Заёмные средства (долгосрочные), RUB', 
               '2018, Заёмные средства (долгосрочные), RUB', '2019, Заёмные средства (долгосрочные), RUB', '2020, Заёмные средства (долгосрочные), RUB',
               '2016, Заёмные средства (краткосрочные), RUB', '2017, Заёмные средства (краткосрочные), RUB',
               '2018, Заёмные средства (краткосрочные), RUB', '2019, Заёмные средства (краткосрочные), RUB',
               '2020, Заёмные средства (краткосрочные), RUB',
               '2016, Краткосрочные обязательства, RUB', '2017, Краткосрочные обязательства, RUB', 
               '2018, Краткосрочные обязательства, RUB', '2019, Краткосрочные обязательства, RUB',
               '2020, Краткосрочные обязательства, RUB',
               '2016, Выручка, RUB', '2017, Выручка, RUB', '2018, Выручка, RUB', '2019, Выручка, RUB', '2020, Выручка, RUB',
               '2016, Прибыль (убыток) от продажи, RUB', '2017, Прибыль (убыток) от продажи, RUB',
               '2018, Прибыль (убыток) от продажи, RUB', '2019, Прибыль (убыток) от продажи, RUB', '2020, Прибыль (убыток) от продажи, RUB', 
               'Итого',
               'ПДЗ 1-30', 'ПДЗ 31-90', 'ПДЗ 91-365', 'ПДЗ более 365']

In [5]:
rename_columns = {'Наименование ДП':'name', 
                  'Макс. ПДЗ за 2019 год, дней':'max_pdz_19',
                            'Сред. ПДЗ за 2019 год, дней':'mean_pdz_19',
                            'Кол-во просрочек свыше 5-ти дней за 2019 год, шт.': 'count5_pdz_19',
                            'Общая сумма ПДЗ свыше 5-ти дней за 2019 год, руб.': 'sum5_pdz_19',
                            'Кол-во раз ПДЗ за 2019 год, шт.': 'countall_pdz_19',
                  'Макс. ПДЗ за 2020 год, дней':'max_pdz_20',
                            'Сред. ПДЗ за 2020 год, дней':'mean_pdz_20',
                            'Кол-во просрочек свыше 5-ти дней за 2020 год, шт.': 'count5_pdz_20',
                            'Общая сумма ПДЗ свыше 5-ти дней за 2020 год, руб.': 'sum5_pdz_20',
                            'Кол-во раз ПДЗ за 2020 год, шт.': 'countall_pdz_20',
                            '2016, Основные средства , RUB': 'oc_t-3',
                            '2017, Основные средства , RUB': 'oc_t-2', 
                            '2018, Основные средства , RUB': 'oc_t-1',
                  '2019, Основные средства , RUB': 'oc_t0',
                  '2020, Основные средства , RUB': 'oc_t1',
                            '2016, Оборотные активы, RUB': 'oborot_t-3', 
                            '2017, Оборотные активы, RUB': 'oborot_t-2',
                            '2018, Оборотные активы, RUB': 'oborot_t-1',
                  '2019, Оборотные активы, RUB': 'oborot_t0',
                  '2020, Оборотные активы, RUB': 'oborot_t1',
                            '2016, Капитал и резервы, RUB': 'kapital_t-3',
                            '2017, Капитал и резервы, RUB': 'kapital_t-2', 
                            '2018, Капитал и резервы, RUB': 'kapital_t-1',
                  '2019, Капитал и резервы, RUB': 'kapital_t0',
                  '2020, Капитал и резервы, RUB': 'kapital_t1',
                            '2016, Заёмные средства (долгосрочные), RUB': 'dolg_zaem_t-3',
                            '2017, Заёмные средства (долгосрочные), RUB': 'dolg_zaem_t-2',
                            '2018, Заёмные средства (долгосрочные), RUB': 'dolg_zaem_t-1',
                  '2019, Заёмные средства (долгосрочные), RUB': 'dolg_zaem_t0',
                  '2020, Заёмные средства (долгосрочные), RUB': 'dolg_zaem_t1',
                            '2016, Заёмные средства (краткосрочные), RUB': 'krat_zaem_t-3',
                            '2017, Заёмные средства (краткосрочные), RUB': 'krat_zaem_t-2',
                            '2018, Заёмные средства (краткосрочные), RUB': 'krat_zaem_t-1',
                  '2019, Заёмные средства (краткосрочные), RUB': 'krat_zaem_t0',
                  '2020, Заёмные средства (краткосрочные), RUB': 'krat_zaem_t1',
                            '2016, Краткосрочные обязательства, RUB': 'krat_liabil_t-3',
                            '2017, Краткосрочные обязательства, RUB': 'krat_liabil_t-2',
                            '2018, Краткосрочные обязательства, RUB': 'krat_liabil_t-1',
                  '2019, Краткосрочные обязательства, RUB': 'krat_liabil_t0',
                  '2020, Краткосрочные обязательства, RUB': 'krat_liabil_t1',
                            '2016, Выручка, RUB': 'revenue_t-3',
                            '2017, Выручка, RUB': 'revenue_t-2',
                            '2018, Выручка, RUB': 'revenue_t-1',
                  '2019, Выручка, RUB': 'revenue_t0',
                  '2020, Выручка, RUB': 'revenue_t1',
                            '2016, Прибыль (убыток) от продажи, RUB': 'profit_t-3',
                            '2017, Прибыль (убыток) от продажи, RUB': 'profit_t-2',
                            '2018, Прибыль (убыток) от продажи, RUB': 'profit_t-1',
                  '2019, Прибыль (убыток) от продажи, RUB': 'profit_t0',
                  '2020, Прибыль (убыток) от продажи, RUB': 'profit_t1',
                  'Итого': 'Itog',
                  'ПДЗ 1-30': 'PDZ30',
                  'ПДЗ 31-90': 'PDZ90', 
                  'ПДЗ 91-365': 'PDZ365', 
                  'ПДЗ более 365': 'PDZ700'
                           }

In [7]:
#function deleted detected year for some columns
def dropyear(data, year):
    data = data.drop([f'oc_t{year}', f'oborot_t{year}', f'kapital_t{year}', f'dolg_zaem_t{year}', 
                      f'krat_zaem_t{year}', f'krat_liabil_t{year}', f'revenue_t{year}', f'profit_t{year}'], axis = 1, inplace = True)

In [8]:
#feat = ['oc_t', 'oborot_t', 'kapital_t', 'dolg_zaem_t', 'krat_zaem_t', 'krat_liabil_t', 'revenue_t', 'profit_t']
#year = -2

def rename(feat, year, data_year=2020):
    rename_col = {}
    if data_year == 2020:
        for f in feat:
            rename_col[f'{f}{year}'] = f'{f}{year-1}'
            rename_col[f'{f}{year+1}'] = f'{f}{year}'
            rename_col[f'{f}{year+2}'] = f'{f}{year+1}'
    elif data_year == 2021:
            for f in feat:
                rename_col[f'{f}{year}'] = f'{f}{year-2}'
                rename_col[f'{f}{year+1}'] = f'{f}{year-1}'
                rename_col[f'{f}{year+2}'] = f'{f}{year}'
    return rename_col

In [9]:
#year = [-3, -2, -1]

def class_revenue(data, year):
    for yr in year:
        data[f'class_rev_t{yr}'] = 0
        data.loc[(data[f'revenue_t{yr}'] > 200000000) & (data[f'revenue_t{yr}'] <= 500000000), f'class_rev_t{yr}'] = 1
        data.loc[(data[f'revenue_t{yr}'] > 500000000) & (data[f'revenue_t{yr}'] <= 700000000), f'class_rev_t{yr}'] = 2
        data.loc[(data[f'revenue_t{yr}'] > 700000000) & (data[f'revenue_t{yr}'] <= 1000000000), f'class_rev_t{yr}'] = 3
        data.loc[(data[f'revenue_t{yr}'] > 1000000000) & (data[f'revenue_t{yr}'] <= 2500000000), f'class_rev_t{yr}'] = 4
        data.loc[data[f'revenue_t{yr}'] > 2500000000, f'class_rev_t{yr}'] = 5

In [12]:
features = ['oc_t', 'kapital_t', 'revenue_t', 'profit_t', 'dolg_zaem_t', 'krat_liabil_t'] #use features to make new features 
f_all =  ['oc_t', 'oborot_t', 'kapital_t', 'dolg_zaem_t', 'krat_zaem_t', 'krat_liabil_t', 'revenue_t', 'profit_t']

def make_df(data, year_3 = [-3, -2, -1], year_2 = [-2, -1], features = features, data_year=2019, f_all = f_all):
    
    if data_year == 2019:
    
        data = data[list(set(data.columns) & set(all_columns))] #only choose columns
        data.rename(columns = rename_columns, inplace = True) #rename choose columns

        fees = []
        for yr in year_3:
            fees.append(f'margin_profit_{yr}') #margin for each year new feature
            fees[0] = pd.DataFrame({fees[0]: round(data[f'profit_t{yr}'] / data[f'revenue_t{yr}'], 3)})
            data[f'margin_profit_{yr}'] = fees[0]
            data[f'margin_profit_{yr}'] = data[f'margin_profit_{yr}'].fillna(0)
            fees.pop(0) 

            fees.append(f'reven_debt{yr}') #debt|ravenue for each year new feature
            fees[0] = pd.DataFrame({fees[0]: round((data[f'dolg_zaem_t{yr}']+data[f'krat_zaem_t{yr}']) / data[f'revenue_t{yr}'], 3)})
            data[f'reven_debt{yr}'] = fees[0]
            data[f'reven_debt{yr}'] = data[f'reven_debt{yr}'].fillna(0)
            fees.pop(0)    

        for yr in year_2:
            for feat in features:
                fees.append(f'growth_{feat}_{yr}')
                fees[0] = pd.DataFrame({fees[0]: round(data[f'{feat}{yr}'] / data[f'{feat}{yr-1}']-1, 3)})
                data[f'growth_{feat}{yr}'] = fees[0]
                data[f'growth_{feat}{yr}'] = data[f'growth_{feat}{yr}'].fillna(0)
                fees.pop(0)
                
        class_revenue(data, year_3) #function new feature class company by ravenue         

        data = data.reindex(sorted(data.columns), axis=1)
        
    elif data_year == 2020:
        data = data[list(set(data.columns) & set(all_columns))] #only choose columns
        data.rename(columns = rename_columns, inplace = True) #rename choose columns   
        
        dropyear(data, -3) #delete not needed year
        data.rename(columns = rename(f_all, -2, data_year), inplace = True) #rename years as current year
        
        fees = []
        for yr in year_3:
            fees.append(f'margin_profit_{yr}') #margin for each year new feature
            fees[0] = pd.DataFrame({fees[0]: round(data[f'profit_t{yr}'] / data[f'revenue_t{yr}'], 3)})
            data[f'margin_profit_{yr}'] = fees[0]
            data[f'margin_profit_{yr}'] = data[f'margin_profit_{yr}'].fillna(0)
            fees.pop(0) 

            fees.append(f'reven_debt{yr}') #debt|ravenue for each year new feature
            fees[0] = pd.DataFrame({fees[0]: round((data[f'dolg_zaem_t{yr}']+data[f'krat_zaem_t{yr}']) / data[f'revenue_t{yr}'], 3)})
            data[f'reven_debt{yr}'] = fees[0]
            data[f'reven_debt{yr}'] = data[f'reven_debt{yr}'].fillna(0)
            fees.pop(0)    

        for yr in year_2:
            for feat in features:
                fees.append(f'growth_{feat}_{yr}')
                fees[0] = pd.DataFrame({fees[0]: round(data[f'{feat}{yr}'] / data[f'{feat}{yr-1}']-1, 3)})
                data[f'growth_{feat}{yr}'] = fees[0]
                data[f'growth_{feat}{yr}'] = data[f'growth_{feat}{yr}'].fillna(0)
                fees.pop(0)
                
        class_revenue(data, year_3) #function new feature class company by ravenue         

        data = data.reindex(sorted(data.columns), axis=1)

        
    elif data_year == 2021:        
        data = data[list(set(data.columns) & set(all_columns))] #only choose columns
        data.rename(columns = rename_columns, inplace = True) #rename choose columns   
        
        dropyear(data, -3) #delete not needed year
        dropyear(data, -2) #delete not needed year
        data.rename(columns = rename(f_all, -1, data_year), inplace = True) #rename years as current year
        
        fees = []
        for yr in year_3:
            fees.append(f'margin_profit_{yr}') #margin for each year new feature
            fees[0] = pd.DataFrame({fees[0]: round(data[f'profit_t{yr}'] / data[f'revenue_t{yr}'], 3)})
            data[f'margin_profit_{yr}'] = fees[0]
            data[f'margin_profit_{yr}'] = data[f'margin_profit_{yr}'].fillna(0)
            fees.pop(0) 

            fees.append(f'reven_debt{yr}') #debt|ravenue for each year new feature
            fees[0] = pd.DataFrame({fees[0]: round((data[f'dolg_zaem_t{yr}']+data[f'krat_zaem_t{yr}']) / data[f'revenue_t{yr}'], 3)})
            data[f'reven_debt{yr}'] = fees[0]
            data[f'reven_debt{yr}'] = data[f'reven_debt{yr}'].fillna(0)
            fees.pop(0)    

        for yr in year_2:
            for feat in features:
                fees.append(f'growth_{feat}_{yr}')
                fees[0] = pd.DataFrame({fees[0]: round(data[f'{feat}{yr}'] / data[f'{feat}{yr-1}']-1, 3)})
                data[f'growth_{feat}{yr}'] = fees[0]
                data[f'growth_{feat}{yr}'] = data[f'growth_{feat}{yr}'].fillna(0)
                fees.pop(0)
                
        class_revenue(data, year_3) #function new feature class company by ravenue

        
    return data
        

In [13]:
df19 = make_df(data_2019, year_3 = [-3, -2, -1], year_2 = [-2, -1], features = features, data_year=2019, f_all = f_all)
df20 = make_df(data_2020, year_3 = [-3, -2, -1], year_2 = [-2, -1], features = features, data_year=2020, f_all = f_all)
df21 = make_df(data_2021, year_3 = [-3, -2, -1], year_2 = [-2, -1], features = features, data_year=2021, f_all = f_all)


In [14]:
df20.columns.difference(df19.columns)

Index(['Itog', 'count5_pdz_20', 'countall_pdz_20', 'max_pdz_20', 'mean_pdz_20',
       'sum5_pdz_20'],
      dtype='object')

In [15]:
df21.columns.difference(df19.columns)

Index(['Itog', 'PDZ30', 'PDZ365', 'PDZ700', 'PDZ90', 'count5_pdz_20',
       'countall_pdz_20', 'max_pdz_20', 'mean_pdz_20', 'sum5_pdz_20'],
      dtype='object')